In [2]:
import os
import random
import sys

import numpy as np
import pandas as pd
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm


sys.path.append(os.path.abspath("../"))
from src.chatbot import RecipeChatbot
from src.config import DEFAULT_QA_MODEL, OPENAI_API_KEY
from src.data import indexing as data_indexing
from src.templates.prompt_templates import QA_GENERATION_TEMPLATE, STANDALONE_CRITIQUE_PROMPT_TEMPLATE, GROUNDNESS_CRITIQUE_PROMPT_TEMPLATE, RELEVANCE_CRITIQUE_PROMPT_TEMPLATE
from src.utils.embeddings import get_all_chroma_embeddings, get_embedding_model
from src.utils.generation import call_llm_openrouter
from src.utils.text_processing import format_dictionary_pairs

For the evaluation task, mainly deepeval library will be used. 

In [ ]:
N_GENERATIONS=10
SIMILARITY_THRESHOLD = 0.925
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["DEEPEVAL_RESULTS_FOLDER"] = "../data/results"
os.environ["%autocall"]

In [7]:
outputs = []
used_questions = {}
embedding_model = get_embedding_model()

Loading embedding model: intfloat/multilingual-e5-large-instruct
Embedding model ready to use.


I believe that the best possible case would be manual creation of the answer-question pairs. However, due to the time constraints it is not possible, therefore we will use an LLM, as described in https://huggingface.co/learn/cookbook/en/rag_evaluation

I am aware of deepevals synthesiser, however, I used custom solution for better flexibility and research on how the QA generation works. 

In [ ]:
chroma_data = get_all_chroma_embeddings(embedding_model)
doc_embeddings = chroma_data['embeddings']
docs = chroma_data['documents']
doc_ids = chroma_data['ids']
doc_metadatas = chroma_data['metadatas']

similarity_matrix = cosine_similarity(doc_embeddings)

doc_groups = []
used_docs = set()

# cluster docs
for i in range(len(docs)):
    if i in used_docs: 
        continue
        
    current_group = [i]
    used_docs.add(i)
    
    similar_indices = np.where(similarity_matrix[i] > SIMILARITY_THRESHOLD)[0]
    for idx in similar_indices:
        if idx != i and idx not in used_docs:
            current_group.append(int(idx))
            used_docs.add(idx)
    
    doc_groups.append(current_group)

print(f"Doc groups: {doc_groups}")

Doc groups: [[0, 6, 20], [1], [2, 30, 79], [3, 4, 10, 26], [5], [7], [8, 35, 55], [9], [11, 16], [12, 14], [13, 22, 71], [15], [17, 37], [18], [19], [21], [23], [24, 44], [25, 31, 68], [27], [28], [29], [32], [33, 97], [34], [36], [38], [39], [40], [41, 60], [42], [43, 52, 56], [45], [46, 62], [47, 54], [48, 61], [49], [50], [51], [53], [57], [58], [59], [63], [64], [65, 83, 85, 89], [66], [67], [69, 80], [70], [72], [73], [74, 76], [75], [77], [78], [81], [82], [84], [86], [87, 88, 96], [90], [91], [92], [93], [94], [95], [98], [99]]


## Generate QA Pairs

In [ ]:
csv_path = f"../data/evaluation/qa_pairs.csv"

N_GENERATIONS = min(N_GENERATIONS, len(doc_groups))

if not os.path.exists(csv_path):
    for doc_group in tqdm(random.sample(doc_groups, N_GENERATIONS)):
        for _ in range(2): 
            n_docs = random.randint(1, len(doc_group))
            
            selected_indices = random.sample(doc_group, n_docs)
            selected_docs = [docs[idx] for idx in selected_indices]
            combined_content = "\n\n".join([doc for doc in selected_docs])

            formatted_qa = format_dictionary_pairs(used_questions)
            QA_PROMTP = QA_GENERATION_TEMPLATE.format(context=combined_content, qa_pairs=formatted_qa)
            output_QA_couple = call_llm_openrouter(QA_PROMTP, DEFAULT_QA_MODEL, {"question": "question", "answer": "answer"})
            used_questions[output_QA_couple["question"]] = output_QA_couple["answer"]
            outputs.append({
                "query": output_QA_couple["question"],
                "expected_output": output_QA_couple["answer"],
                "actual_output": "",  
                "context": combined_content,  
                "retrieval_context": ""
            })
            
            # print(f"Question: {output_QA_couple['question']}")
            # print(f"Answer: {output_QA_couple['answer']}")

    qa_df = pd.DataFrame(outputs)
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    qa_df.to_csv(csv_path, index=False)
    print(f"Saved {len(outputs)} QA pairs to {csv_path}")

  0%|          | 0/10 [00:00<?, ?it/s]

['question', 'answer']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': 'Your task is to generate a question and its corresponding answer based on the provided recipe context. \n\n# Instructions:\n- Use czech language for both questions and answers.\n\n## Question Guidelines\n- The question must be answerable given the information from the context. DO NOT create questions that require external knowledge.\n- Create questions that reflect realistic user scenarios:\n  - COOKING PHASE: Questions about techniques, ingredients, or clarification of steps. Be creative about what kind of questions a user might ask while cooking (e.g., "Kolik [ingredience] potřebuji pro [část receptu]?", "Můžu použít [X] místo [Y] při přípravě tohoto kroku?")\n  - SELECTION PHASE: Questions about dietary fit, ingredient requirements, general dish selection, cuisine characteristics, what dishes can be made with certain ingredients, or recipes that fit user preferences (e.g., "Mám 

 10%|█         | 1/10 [00:01<00:10,  1.17s/it]

Question: Co je potřeba mít na přípravu holandské omáčky?
Answer: Pro přípravu holandské omáčky je třeba mít žloutky.
['question', 'answer']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': 'Your task is to generate a question and its corresponding answer based on the provided recipe context. \n\n# Instructions:\n- Use czech language for both questions and answers.\n\n## Question Guidelines\n- The question must be answerable given the information from the context. DO NOT create questions that require external knowledge.\n- Create questions that reflect realistic user scenarios:\n  - COOKING PHASE: Questions about techniques, ingredients, or clarification of steps. Be creative about what kind of questions a user might ask while cooking (e.g., "Kolik [ingredience] potřebuji pro [část receptu]?", "Můžu použít [X] místo [Y] při přípravě tohoto kroku?")\n  - SELECTION PHASE: Questions about dietary fit, ingredient requirements, general dish selection, cuisin

 20%|██        | 2/10 [00:02<00:10,  1.35s/it]

...

Question: Proč se má strouhat jen žlutá část citrónové kůry?
Answer: Žlutá část kůry citronu se používá, protože bílá část je hořká.
['question', 'answer']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': 'Your task is to generate a question and its corresponding answer based on the provided recipe context. \n\n# Instructions:\n- Use czech language for both questions and answers.\n\n## Question Guidelines\n- The question must be answerable given the information from the context. DO NOT create questions that require external knowledge.\n- Create questions that reflect realistic user scenarios:\n  - COOKING PHASE: Questions about techniques, ingredients, or clarification of steps. Be creative about what kind of questions a user might ask while cooking (e.g., "Kolik [ingredience] potřebuji pro [část receptu]?", "Můžu použít [X] místo [Y] při přípravě tohoto kroku?")\n  - SELECTION PHASE: Questions about dietary fit, ingredient requirements, general dish se

 90%|█████████ | 9/10 [00:09<00:01,  1.05s/it]

Question: Jak mám připravit okurku do okurkového salátu?
Answer: Okurku je třeba předtím umýt, osušit a nakrájet na tenká půlkolečka nebo kolečka.
['question', 'answer']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': 'Your task is to generate a question and its corresponding answer based on the provided recipe context. \n\n# Instructions:\n- Use czech language for both questions and answers.\n\n## Question Guidelines\n- The question must be answerable given the information from the context. DO NOT create questions that require external knowledge.\n- Create questions that reflect realistic user scenarios:\n  - COOKING PHASE: Questions about techniques, ingredients, or clarification of steps. Be creative about what kind of questions a user might ask while cooking (e.g., "Kolik [ingredience] potřebuji pro [část receptu]?", "Můžu použít [X] místo [Y] při přípravě tohoto kroku?")\n  - SELECTION PHASE: Questions about dietary fit, ingredient requirements, g

100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

Question: Jak mám postupovat, když se mi omáčka přesolí?
Answer: Pokud se omáčka přesolí, je důležité sůl důkladně promíchat, aby se rozpustila, a pak teprve ochutnat a případně dosolit.
Saved 10 QA pairs to ../data/evaluation/qa_pairs.csv


### Critique Agents to Filter out Bad QAs

In [5]:
print("Generating critique for each QA couple...")
csv_path = f"../data/evaluation/qa_pairs.csv"

outputs = pd.read_csv(csv_path).to_dict("records")

for output in outputs:
    output["groundedness_rating"] = 0
    output["relevance_rating"] = 0
    output["standalone_rating"] = 0
    output["groundedness_eval"] = ""
    output["relevance_eval"] = ""
    output["standalone_eval"] = ""

for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm_openrouter(
            GROUNDNESS_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"], context=output["context"]),
            json_schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}        
            ),
        "relevance": call_llm_openrouter(
            
            RELEVANCE_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"]),
            json_schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}
        ),
        "standalone": call_llm_openrouter(
            STANDALONE_CRITIQUE_PROMPT_TEMPLATE.format(question=output["query"]),
            json_schema={"evaluation": "Provide a brief explanation of why you assigned this rating.", "total_rating": "Provide a number from 1 to 5."}
        ),
    }
    
    output["groundedness_rating"] = int(evaluations["groundedness"]["total_rating"])
    output["relevance_rating"] = int(evaluations["relevance"]["total_rating"])
    output["standalone_rating"] = int(evaluations["standalone"]["total_rating"])
    output["groundedness_eval"] = evaluations["groundedness"]["evaluation"]
    output["relevance_eval"] = evaluations["relevance"]["evaluation"]
    output["standalone_eval"] = evaluations["standalone"]["evaluation"]
    
non_filtered_qa_df = pd.DataFrame(outputs)
non_filtered_csv_path = "../data/evaluation/qa_pairs_eval_non_filtered.csv"
non_filtered_qa_df.to_csv(non_filtered_csv_path, index=False)

filtered_outputs = [
output for output in outputs 
if (output["groundedness_rating"] >= 4 and 
    output["relevance_rating"] >= 4 and 
    output["standalone_rating"] >= 4)
]

print(f"Filtered from {len(outputs)} to {len(filtered_outputs)} high-quality QA pairs")

filtered_qa_df = pd.DataFrame(filtered_outputs)
filtered_csv_path = "../data/evaluation/qa_pairs_filtered.csv"
filtered_qa_df.to_csv(filtered_csv_path, index=False)


Generating critique for each QA couple...


  0%|          | 0/10 [00:00<?, ?it/s]

['evaluation', 'total_rating']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': "\nYou will be given a question and a context.  \nYour task is to evaluate how well the question can be answered **exclusively using the given context.**  \n\n### **Rating Scale (1-5):**  \n- **1** → The question is **not answerable** at all based on the context.  \n- **5** → The question is **fully and unambiguously answerable** using the context alone.  \n\n**Question:** Co je potřeba mít na přípravu holandské omáčky?  \n**Context:** recipe_name: Chřest s holandskou omáčkou\nauthor_name: Roman Vaněk\ningredients: Bílý vinný ocet, Brambory typ A, B, C, Čerstvě drcený pepř, Chřest bílý, Chřest zelený, Citronová šťáva, Cukr krupice, Kuchyňský provázek, Máslo, Suché bílé víno, Sůl, Žloutek\nsteps: ['Bílý chřest oloupeme a odřízneme dřevnatý konec. Zelený chřest neloupeme, jen z jeho stonků odřízneme lístky. Bílý a zelený chřest svážeme zvlášť po porcích kuchyňským provázkem.',

 10%|█         | 1/10 [00:03<00:34,  3.87s/it]

['evaluation', 'total_rating']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': "\nYou will be given a question and a context.  \nYour task is to evaluate how well the question can be answered **exclusively using the given context.**  \n\n### **Rating Scale (1-5):**  \n- **1** → The question is **not answerable** at all based on the context.  \n- **5** → The question is **fully and unambiguously answerable** using the context alone.  \n\n**Question:** Jaké ingredience jsou potřeba na přípravu těsta na halušky?  \n**Context:** recipe_name: Bryndzové halušky\nauthor_name: Roman Vaněk\ningredients: Brambory varný typ B, Bryndza, Cibule, Mléko, Polohrubá mouka, Sádlo, Slanina, Smetana ke šlehání (min. 31%), Sůl, Vejce, Vhodné pivo, Zakysaná smetana\nsteps: ['Brambory nastrouháme najemno. Dáme do mísy a vymačkáme z nich vodu. Přidáme mouku, vejce a sůl a vymícháme těsto. Ve velkém hrnci přivedeme k varu větší množství vody.', 'Část těsta na halušky rozetřeme

...

['evaluation', 'total_rating']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': "\nYou will be given a question and a context.  \nYour task is to evaluate how well the question can be answered **exclusively using the given context.**  \n\n### **Rating Scale (1-5):**  \n- **1** → The question is **not answerable** at all based on the context.  \n- **5** → The question is **fully and unambiguously answerable** using the context alone.  \n\n**Question:** Jak mám připravit okurku do okurkového salátu?  \n**Context:** recipe_name: Tzatziky\nauthor_name: Roman Vaněk\ningredients: Bílý řecký jogurt, Čerstvý kopr, Česnek, Citronová šťáva, Extra panenský olivový olej, Salátová okurka, Sůl\nsteps: ['Česnek podélně rozřízneme a vyndáme tuhý klíček, který vyhodíme. Česnek najemno nakrájíme.', 'Okapanou okurku vymačkáme, smícháme v misce s jogurtem, česnekem, koprem, citronovou šťávou a olivovým olejem. Dáme na 2 hodiny vychladit do lednice. Před podáváním promíchám

 90%|█████████ | 9/10 [00:30<00:03,  3.19s/it]

['evaluation', 'total_rating']
{'model': 'google/gemini-2.0-flash-001', 'messages': [{'role': 'user', 'content': "\nYou will be given a question and a context.  \nYour task is to evaluate how well the question can be answered **exclusively using the given context.**  \n\n### **Rating Scale (1-5):**  \n- **1** → The question is **not answerable** at all based on the context.  \n- **5** → The question is **fully and unambiguously answerable** using the context alone.  \n\n**Question:** Jak mám postupovat, když se mi omáčka přesolí?  \n**Context:** recipe_name: Koprová omáčka\nauthor_name: Roman Vaněk\ningredients: Bobkový list, Celer, Černý pepř celý, Čerstvý kopr, Česnek, Cibule, Cukr krupice, Hladká mouka, Houskové knedlíky, Hovězí plec vcelku, Hovězí vývar, Máslo, Mrkev, Nové koření celé, Ocet, Petržel, Smetana ke šlehání (min. 31%), Sůl, Vhodné pivo\nsteps: ['Chcete-li omáčku dokonale hladkou, můžete ji před přidáním nakrájeného kopru přecedit přes jemné síto.', 'Do omáčky můžete dát

100%|██████████| 10/10 [00:33<00:00,  3.34s/it]

Filtered from 10 to 6 high-quality QA pairs


## Generate Actual Output with RAG system

Let's generate actual outputs using our RAG system for the questions: 

In [6]:
chatbot = RecipeChatbot(embedding_model)
qa_df = pd.read_csv(filtered_csv_path)

for idx, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
    query = row["query"]
    actual_output = chatbot.chat(query, False)
    print(actual_output)
    qa_df.loc[idx, "actual_output"] = actual_output["answer"]
    qa_df.loc[idx, "retrieval_context"] = actual_output["context"]

qa_df.to_csv(csv_path, index=False)

NameError: name 'embedding_model' is not defined

In [ ]:
owvnwof

In [ ]:
dataset = EvaluationDataset()

dataset.add_test_cases_from_csv_file(
    file_path=filtered_csv_path,
    input_col_name="query",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    retrieval_context_col_name="retrieval_context"
    )

metric = AnswerRelevancyMetric(
    model="gpt-4o"
)

print(dataset)

EvaluationDataset(test_cases=[LLMTestCase(input='Co je potřeba mít na přípravu holandské omáčky?', actual_output='Na přípravu holandské omáčky budete potřebovat následující ingredience z receptu "Chřest s holandskou omáčkou" od Romana Vaňka:\n\n- Žloutky\n- Máslo\n- Bílý vinný ocet\n\nPokud budete mít zájem o podrobné instrukce k přípravě holandské omáčky nebo celého receptu, dejte mi vědět! Dobrou chuť!', expected_output='Pro přípravu holandské omáčky je třeba mít žloutky.', context=["recipe_name: Chřest s holandskou omáčkou\nauthor_name: Roman Vaněk\ningredients: Bílý vinný ocet, Brambory typ A, B, C, Čerstvě drcený pepř, Chřest bílý, Chřest zelený, Citronová šťáva, Cukr krupice, Kuchyňský provázek, Máslo, Suché bílé víno, Sůl, Žloutek\nsteps: ['Bílý chřest oloupeme a odřízneme dřevnatý konec. Zelený chřest neloupeme, jen z jeho stonků odřízneme lístky. Bílý a zelený chřest svážeme zvlášť po porcích kuchyňským provázkem.', 'Chřest oloupejte škrabkou na chřest nebo škrabkou na brambor

In [ ]:
evaluate(test_cases=dataset.test_cases, metrics=[metric])    

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 10 test case(s) in parallel: |██████████|100% (10/10) [Time Taken: 00:20,  2.03s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.8, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.80 because the main content effectively addresses the sugar usage in relation to the cherry compote. However, a pleasant wish was included that doesn't contribute to the discussion about sugar, preventing a higher score., error: None)

For test case:

  - input: Mám použít méně cukru, pokud použiji višňový kompot?
  - actual output: Ano, pokud použijete višňový kompot místo mražených višní při přípravě Višňového crumble, doporučuje se do směsi přimíchat méně cukru. Kompot už je obvykle oslazený, takže by dezert mohl být příliš sladký, pokud byste přidali stejné množství cukru jako v originálním receptu. Dobrou chuť!
  - expected output: Do třešňové směsi můžete přidat méně cukru, pokud použijete višňový kompot.
  - context: ["recipe_name: Višňový crumble\nauthor_name: Roman Vaněk\ningredients: Cukr krupice, Hořká čokoláda, Hrubá mouka, Káva vhodná k tom

Results saved in ../data/results as 20250311_012736

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_2', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.8, reason="The score is 0.80 because the main content effectively addresses the sugar usage in relation to the cherry compote. However, a pleasant wish was included that doesn't contribute to the discussion about sugar, preventing a higher score.", strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.005380000000000001, verbose_logs='Statements:\n[\n    "Použijte višňový kompot místo mražených višní při přípravě Višňového crumble.",\n    "Doporučuje se do směsi přimíchat méně cukru.",\n    "Kompot už je obvykle oslazený.",\n    "Dezert by mohl být příliš sladký s původním množstvím cukru.",\n    "Dobrou chuť!"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "idk",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        